# Cisco DNA Center API

## How to get a token
All Cisco DNA Center API calls are authenticated.  Here is a code snippet to get a valid token.

The "dnac_config.py" file contains variables for the controller and the username and password


In [ ]:
from dnac_config import DNAC, DNAC_PORT, DNAC_USER, DNAC_PASSWORD
import requests
from requests.auth import HTTPBasicAuth
requests.packages.urllib3.disable_warnings()


In [ ]:
login_url = "https://{0}:{1}/api/system/v1/auth/token".format(DNAC, DNAC_PORT)
result = requests.post(url=login_url, auth=HTTPBasicAuth(DNAC_USER, DNAC_PASSWORD), verify=False)
result.raise_for_status()
token = result.json()["Token"]

In [ ]:
print (token)

## Using some predefined functions


In [ ]:
from __future__ import print_function
import sys
import json
from util import get_url, deploy_and_wait, post_and_wait

In [ ]:
response = get_url("dna/intent/api/v1/network-device")

In [ ]:
print(json.dumps(response))

In [ ]:
data = ", ".join([ "{}:{}".format(dev['managementIpAddress'],dev['platformId'])
                  for dev in response['response']])

In [ ]:
print (data)

## Templates

In [ ]:
templates = get_url("dna/intent/api/v1/template-programmer/template")

In [ ]:
template = [ t for t in templates if t['projectName'] == "switching" and t['name'] == "interface-des"][0]

In [ ]:
print(json.dumps(template, indent=2))

In [ ]:
templateId= template['templateId']


In [ ]:
print(templateId)

In [ ]:
latest_id = 0
max = 0
for v in template['versionsInfo']:
    if int(v['version']) > max:
        max = int(v['version'])
        latest_id = v['id']
print(latest_id)

In [ ]:
templateData = get_url("dna/intent/api/v1/template-programmer/template/{}".format(templateId))

In [ ]:
print(json.dumps(templateData, indent=2))

### Applying the Template

In [ ]:
payload = {
    "templateId": latest_id,
    "forcePushTemplate" : False,
    "targetInfo": [
     {

        "id": "10.10.20.81",
        "type": "MANAGED_DEVICE_IP",
        "params": {"description": "my Desc", "interface":"g1/0/12"}
        }
     ]
    }

In [ ]:
deployment = deploy_and_wait("dna/intent/api/v1/template-programmer/template/deploy", payload)

In [ ]:
print(json.dumps(deployment,indent=2))